In [1]:
# 首先 import 必要的模块
import pandas as pd 
import numpy as np

from sklearn.model_selection import GridSearchCV

#竞赛的评价指标为logloss
from sklearn.metrics import log_loss  

from matplotlib import pyplot
import seaborn as sns
%matplotlib inline

In [2]:
# 读取数据
# path to where the data lies
dpath = '../data/'
train = pd.read_csv(dpath +"incomeClassTrainMapped.csv")
train.head()

,Unnamed: 0,Age,workclass,People,Education,EduNum,Marriage,WorkType,Family,Race,Sex,capitalgain,capitalloss,hours,Country,Income
0,0,39,0,77516,0,13,0,0,0,0,0,2174,0,40,0,0
1,1,50,1,83311,0,13,1,1,1,0,0,0,0,13,0,0
2,2,38,2,215646,1,9,2,2,0,0,0,0,0,40,0,0
3,3,53,2,234721,2,7,1,2,1,1,0,0,0,40,0,0
4,4,28,2,338409,0,13,1,3,2,1,1,0,0,40,1,0


In [3]:
y_train = train["Income"]

train = train.drop(["Income"], axis=1)
X_train = np.array(train)
X_train

array([[    0,    39,     0, ...,     0,    40,     0],
       [    1,    50,     1, ...,     0,    13,     0],
       [    2,    38,     2, ...,     0,    40,     0],
       ...,
       [32558,    58,     2, ...,     0,    40,     0],
       [32559,    22,     2, ...,     0,    20,     0],
       [32560,    52,     6, ...,     0,    40,     0]])

In [5]:
from sklearn.model_selection import train_test_split

# 将训练集合拆分成训练集和校验集，在校验集上找到最佳的模型超参数（PCA的维数）
X_train_part, X_val, y_train_part, y_val = train_test_split(X_train,y_train, train_size = 0.8,random_state = 0)

/usr/local/python36/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [6]:
#拆分后的训练集和校验集的样本数目
print(X_train_part.shape)
print(X_val.shape)

(26048, 15)
(6513, 15)


In [11]:
from sklearn.cluster import MiniBatchKMeans
import time
from sklearn import metrics

# 一个参数点（聚类数据为K）的模型，在校验集上评价聚类算法性能
def K_cluster_analysis(K, X_train, y_train, X_val, y_val):
    start = time.time()
    
    print("K-means begin with clusters: {}".format(K));
    
    #K-means,在训练集上训练
    mb_kmeans = MiniBatchKMeans(n_clusters = K)
    mb_kmeans.fit(X_train)
    
    # 在训练集和测试集上测试
    #y_train_pred = mb_kmeans.fit_predict(X_train)
    y_val_pred = mb_kmeans.predict(X_val)
    
    #以前两维特征打印训练数据的分类结果
    #plt.scatter(X_train[:, 0], X_train[:, 1], c=y_pred)
    #plt.show()

    # K值的评估标准
    #常见的方法有轮廓系数Silhouette Coefficient和Calinski-Harabasz Index
    #这两个分数值越大则聚类效果越好
    #CH_score = metrics.calinski_harabaz_score(X_train,mb_kmeans.predict(X_train))
    CH_score = metrics.silhouette_score(X_train,mb_kmeans.predict(X_train))
    
    #也可以在校验集上评估K
    v_score = metrics.v_measure_score(y_val, y_val_pred)
    
    end = time.time()
    print("CH_score: {}, time elaps:{}".format(CH_score, int(end-start)))
    print("v_score: {}".format(v_score))
    
    return CH_score,v_score

In [13]:
# 设置超参数（聚类数目K）搜索范围
Ks = [10, 20, 30,40,50]
CH_scores = []
v_scores = []
for K in Ks:
    ch,v = K_cluster_analysis(K, X_train_part, y_train_part, X_val, y_val)
    CH_scores.append(ch)
    v_scores.append(v)

K-means begin with clusters: 10


MemoryError: 

In [ ]:
# 绘制不同PCA维数下模型的性能，找到最佳模型／参数（分数最高）
plt.plot(Ks, np.array(CH_scores), 'b-')